In [1]:
import sys

sys.path.insert(0, "../src")

In [2]:
import pandas as pd
from pathlib import Path

DATA_FOLDER = Path("../data")
MODELS_FOLDER = Path("../models/regression")

collisions = pd.read_pickle(DATA_FOLDER / "final_dataset_test.pkl")
collisions = collisions[collisions["has_volume_meas"]]

In [3]:
X_test = collisions.drop(
    columns=[
        "physicalid",
        "geometry",
        "has_parking_meters",
        "collision_rate",
        "collision_rate_per_length",
        "has_volume_meas",
    ]
)

y_test = collisions["collision_rate_per_length"]

In [4]:
from sklearn.metrics import mean_squared_error
from model_helpers.feature_transformers import LogarithmTransfomer, InverseTransformer
from model_helpers.model_loader import ModelLoader

In [5]:
linear_reg = ModelLoader().load("linear")
forest_reg = ModelLoader().load("random_forest")
xgboost = ModelLoader().load("xgboost")

In [6]:
y_pred = linear_reg.predict(X_test)
mean_squared_error(y_test, y_pred)

4.363916606998886e-07

In [7]:
y_pred = forest_reg.predict(X_test)
mean_squared_error(y_test, y_pred)

3.873500755159358e-07

In [8]:
y_pred = xgboost.predict(X_test)
mean_squared_error(y_test, y_pred)

3.7921756243711133e-07